<a href="https://colab.research.google.com/github/yt1970/ai_engineering_03_homework/blob/main/ai_engineering_03_%E4%BB%BB%E6%84%8F%E5%AE%BF%E9%A1%8CT4%EF%BC%88%E6%8F%90%E5%87%BA%E7%94%A8%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



このノートブックは、「AIエンジニアリング講座」DAY３任意宿題用RAG検証ノートブックです。
GPUは「T4」で行っています。
モデルは、gemma-2-2b-jpn-itを使用しています。

参照文書は防衛省令和７年度予算の概要（令和７年４月２日）のPDF版です。

[https://www.mod.go.jp/j/budget/yosan_gaiyo/fy2025/yosan_20250402.pdf](https://www.mod.go.jp/j/budget/yosan_gaiyo/fy2025/yosan_20250402.pdf)

---

# 宿題実施の方針

1. **ベースラインモデルの評価**  
   素のモデルで回答を生成し、最新の情報に対しては正しく回答できないことを確認します。。

2. **参照文書のテキスト化**  
   参照文書をPyPDF2でテキスト化します。

3. **文書整形なしでのページごとのチャンク化**  
   文書整形なしでぺーじごとにチャンク化し、回答を生成させます。

4. **文書整形ありでのページごとのチャンク化**  
   無駄な空白や改行の削除、意味のある改行の追加などにより回答の精度向上を図ります。

5. **回答精度の測定方法**
    googleGeminiのAPIを利用して、Geminiに評価指標を与え、点数で評価します。  
   

## 質問事項

  宿題の要件に従い、参照文書に関するテーマから５つを質問事項として設け、模範解答も合わせて設定しました。

  なお、５つの質問のうち、１つは参照文書をみても回答ができない事項であり、その旨をきちんと回答できるかどうかも評価できるようにしました。


```
[
  {
    "質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。",
    "模範解答": "令和7年度の防衛費の予算総額は、歳出ベースで8兆4,748億円、契約ベースで8兆4,332億円です。\n\nその主要な内訳は以下の通りです。（歳出ベース）\n\n* 装備品費：3兆1,714億円\n* 需品費・施設費：1兆4,918億円\n* 人件・糧食費等：2兆9,186億円\n* 研究開発費：8,176億円\n* 施設整備費：7,586億円\n* その他：3,168億円",
    "出典": "1ページ「１．歳出」"
  },
  {
    "質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。",
    "模範解答": "令和7年度の防衛力整備計画で重視されている7つの分野は以下の通りです。\n\n1.  スタンド・オフ防衛能力の強化\n2.  統合防空ミサイル防衛能力の強化\n3.  無人アセット防衛能力の強化\n4.  宇宙・サイバー・電磁波領域における能力の強化\n5.  情報機能の強化\n6.  輸送能力の強化\n7.  持続性・強靭性の強化\n\n具体的な取り組みの例については、資料の2ページ「II．主要経費 (1) 重点的な整備・強化を行う主要分野」に記載されています。",
    "出典": "2ページ「II．主要経費 (1) 重点的な整備・強化を行う主要分野」"
  },
  {
    "質問": "過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。",
    "模範解答": "資料には令和7年度の予算概要のみが記載されており、過去5年間の防衛費の推移に関する具体的な数値データは含まれていません。そのため、この資料のみで過去5年間の防衛費の推移とGDP比での変化を説明することはできません。",
    "出典": "資料全体（過去のデータに関する記載なし）"
  },
  {
    "質問": "令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。",
    "模範解答": "令和7年度の防衛予算において、自衛官等の給与・手当等の見直しの予算額は**168億円**です。主な項目は、自衛官等の生活・勤務環境の特殊性等に関する処遇改善策、自衛隊の任務の特殊性に関する処遇改善策などです。",
    "出典": "42ページ「Ⅳ 共通基盤」"
  },
  {
    "質問": "令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？",
    "模範解答": "令和7年度防衛省予算で取得を計画しているのは、固定翼哨戒機（Ｐ－１）**2機**（848億円）と回転翼哨戒機（ＳＨ－６０Ｌ）**2機**（293億円）であり、合計は**4機**、**1,141億円**です。",
    "出典": "26ページ「Ⅲ 主要事項」"
  }
]
```







## 扱うモデル

「google/gemma-2-2b-jpn-it」を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# ドライブの設定
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
datafolder = "/content/drive/MyDrive/Colab Notebooks/AIengineering2025_colab/DAY3/data/"
pdfname = "yosan_20250402.pdf"
pdfpath = datafolder + pdfname

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/yt1970/lecture-ai-engineering.git

Cloning into 'lecture-ai-engineering'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99 (from 2)
Receiving objects: 100% (99/99), 93.77 KiB | 4.69 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Gemma2)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

llm_model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

llm_model = AutoModelForCausalLM.from_pretrained(
            llm_model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

tokenizer_config.json:   0%|          | 0.00/46.9k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
# QAファイルの読み込み
# JSONファイルから質問を読み込む
import json
questions_file = datafolder + "Q&A.json" # ファイル名を適切に変更してください
try:
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions_data = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{questions_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{questions_file}' のJSON形式が不正です。")
    exit()

prompt_questions = ""
original_questions = []
for item in questions_data:
    prompt_questions += f'{{"質問": "{item["質問"]}"}},\n'
    original_questions.append(item["質問"])
prompt_questions = "[" + prompt_questions.rstrip(',\n') + "]"

print(prompt_questions)

[{"質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。"},
{"質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。"},
{"質問": "過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。"},
{"質問": "令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。"},
{"質問": "令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？"}]


In [ ]:
question_json_file = datafolder + "question.json"
parsed_response = json.loads(prompt_questions)
with open(question_json_file, 'w', encoding='utf-8') as f:
  json.dump(parsed_response, f, ensure_ascii=False, indent=2)

# 1 ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

### 1.1 回答内容の生成

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# モデルとトークナイザーの準備
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

def generate_output(query):
    messages = [
        {"role": "user", "content": query},
    ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        do_sample=False,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)



prompt_questions = ""
original_questions = []
for item in questions_data:
    prompt_questions += f'{{"質問": "{item["質問"]}"}},\n'
    original_questions.append(item["質問"])
prompt_questions = "[" + prompt_questions.rstrip(',\n') + "]"

prompt = f"""以下の質問に、質問と回答のペアをJSON形式で出力してください。

{prompt_questions}

出力形式:
[
  {{
    "質問": "質問1",
    "回答": "回答1"
  }},
  {{
    "質問": "質問2",
    "回答": "回答2"
  }},
  ...
]"""

response = generate_output(prompt)
print("生成されたレスポンス:\n", response)

output_json_file = datafolder + "llm_responses.json"

try:
    parsed_response = json.loads(response)
    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(parsed_response, f, ensure_ascii=False, indent=2)
    print(f"\nLLMのレスポンスは '{output_json_file}' に保存されました。")

    for i, item in enumerate(parsed_response):
        print(f"質問：\n{original_questions[i]}\n回答：\n{item['回答']}\n")

except json.JSONDecodeError as e:
    print(f"JSONデコードエラー: {e}")
    print(f"生成されたレスポンス:\n{response}")
except IndexError:
    print("エラー: 生成された回答の数と質問の数が一致しません。")
    print(f"生成されたレスポンス:\n{response}")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

生成されたレスポンス:
 user
以下の質問に、質問と回答のペアをJSON形式で出力してください。

[{"質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。"},
{"質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。"},
{"質問": "過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。"},
{"質問": "令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。"},
{"質問": "令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？"}]

出力形式:
[
  {
    "質問": "質問1",
    "回答": "回答1"
  },
  {
    "質問": "質問2",
    "回答": "回答2"
  },
  ...
]
model
[
  {
    "質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。",
    "回答": "令和7年度の防衛費の予算総額は、約10兆円です。主要な内訳は、 \
  * **装備品**:  最新鋭の戦闘機や艦船の購入や整備 \
  * **人材**:  自衛官の給与・手当の増加や、防衛関連の専門家育成 \
  * **運用**:  防衛施設の維持・管理や、防衛システムの運用 \
  * **研究開発**:  新しい技術の開発や、既存技術の改良 \
  * **情報収集**:  敵の動きや状況を把握するための情報収集 \
  * **国際協力**:  他の国と協力して、国際的な安全保障の維持"
  },
  {
    "質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。",
    "回答": "令和7年度の防衛力整備計画では、以下の7つの分野が重視されています。\
  * **戦闘能力の向上**:  最新鋭の戦闘機や艦船の導入、戦闘システムの強化\
  * **情報収集・分析**:  より高度な情報

In [ ]:
print(response)

user
以下の質問に、質問と回答のペアをJSON形式で出力してください。

[{"質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。"},
{"質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。"},
{"質問": "過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。"},
{"質問": "令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。"},
{"質問": "令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？"}]

出力形式:
[
  {
    "質問": "質問1",
    "回答": "回答1"
  },
  {
    "質問": "質問2",
    "回答": "回答2"
  },
  ...
]
model
[
  {
    "質問": "令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。",
    "回答": "令和7年度の防衛費の予算総額は、約10兆円です。主要な内訳は、 \
  * **装備品**:  最新鋭の戦闘機や艦船の購入や整備 \
  * **人材**:  自衛官の給与・手当の増加や、防衛関連の専門家育成 \
  * **運用**:  防衛施設の維持・管理や、防衛システムの運用 \
  * **研究開発**:  新しい技術の開発や、既存技術の改良 \
  * **情報収集**:  敵の動きや状況を把握するための情報収集 \
  * **国際協力**:  他の国と協力して、国際的な安全保障の維持"
  },
  {
    "質問": "令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。",
    "回答": "令和7年度の防衛力整備計画では、以下の7つの分野が重視されています。\
  * **戦闘能力の向上**:  最新鋭の戦闘機や艦船の導入、戦闘システムの強化\
  * **情報収集・分析**:  より高度な情報収集と分析能力の向上\
 

モデルの出力が変な形になったので、整形し直して、JSONファイルに保存しました。

保存ファイル名は以下の通りです。

llm_responses.json



In [ ]:
import json
import re

try:
    model_start = response.find("model\n")
    if model_start == -1:
        raise ValueError("response 内に 'model\\n' が見つかりませんでした。")

    json_part = response[model_start + len("model\n"):]

    # バックスラッシュ+空白の削除（例: " \\n" や " \\"）
    json_part = re.sub(r'\\\s*', '', json_part)

    # JSONとして読み込む
    parsed_response = json.loads(json_part)

    # 保存
    output_json_file = datafolder + "llm_responses.json"
    with open(output_json_file, 'w', encoding='utf-8') as f:
        json.dump(parsed_response, f, ensure_ascii=False, indent=2)

    print(f"\nLLMのレスポンスは '{output_json_file}' に保存されました。")
    for i, item in enumerate(parsed_response):
        print(f"質問：\n{item['質問']}\n回答：\n{item['回答']}\n")

except json.JSONDecodeError as e:
    print(f"JSONの読み込みに失敗しました: {e}")
except Exception as e:
    print(f"エラーが発生しました: {e}")


LLMのレスポンスは '/content/drive/MyDrive/Colab Notebooks/AIengineering2025_colab/DAY3/data/llm_responses.json' に保存されました。
質問：
令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。
回答：
令和7年度の防衛費の予算総額は、約10兆円です。主要な内訳は、 * **装備品**:  最新鋭の戦闘機や艦船の購入や整備 * **人材**:  自衛官の給与・手当の増加や、防衛関連の専門家育成 * **運用**:  防衛施設の維持・管理や、防衛システムの運用 * **研究開発**:  新しい技術の開発や、既存技術の改良 * **情報収集**:  敵の動きや状況を把握するための情報収集 * **国際協力**:  他の国と協力して、国際的な安全保障の維持

質問：
令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。
回答：
令和7年度の防衛力整備計画では、以下の7つの分野が重視されています。* **戦闘能力の向上**:  最新鋭の戦闘機や艦船の導入、戦闘システムの強化* **情報収集・分析**:  より高度な情報収集と分析能力の向上* **サイバーセキュリティ**:  サイバー攻撃に対する防御能力の強化* **人材育成**:  防衛関連の専門家育成、技術革新への対応* **防衛システムの運用**:  運用効率の向上、システムの信頼性向上* **国際協力**:  国際的な安全保障の維持、協力体制の強化* **災害対応**:  災害発生時の迅速な対応能力の向上

質問：
過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。
回答：
過去5年間の日本の防衛費の推移は以下の通りです。* 2018年度：約9兆円* 2019年度：約10兆円* 2020年度：約11兆円* 2021年度：約12兆円* 2022年度：約13兆円GDP比での変化としては、2018年から2022年にかけて、防衛費はGDP比で約1.5％増加しました。これは、安全保障の重要性と、国際的な安全保障環境の変化に対応するためです。

質問：
令和7年度の防衛予算におい

### 1.2 回答内容の評価方法

- 数値的な評価を見てみます。演習で講師が用いたRagasのAnswer Accuracy的な評価指標を以下のように作成しました。これを利用して数値的な評価を行います。
評価指標の作成にあたっては、Ggeminiに壁打ちを致しました。


```
評価スケール：
    0: 全く不正確。回答の全てまたは主要な部分が誤っており、質問に対して全く的外れな情報を提供している。
    1: ほぼ不正確。回答の大部分が不正確であるか、誤った情報に基づいている。わずかに正しい情報が含まれている程度。
    2: 部分的に正確。回答には正確な情報も含まれているが、重要な誤りや不正確な点、または誤解を招く可能性のある記述が含まれている。
    3: ほぼ正確。回答の大部分は正確であり、質問に対して適切に答えている。しかし、細部にわずかな不正確さ、曖昧さ、または重要でない情報の欠落が見られる場合がある。
    4: 完全に正確。回答は質問に対して適切かつ網羅的に答えており、提示された情報に誤りや不確かな点は一切なく、事実に基づいている。コンテキストとも完全に一致している。
```



- 無料で利用できるので、今回はGeminiAPIを利用してgemini-2.0-flashで評価しました。

必要なライブラリーをインストールします。

In [ ]:
!pip install langchain
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


評価のテストを行います。

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# **ここにあなたの Google AI Studio の API キーを入力してください**
GOOGLE_API_KEY = ""

# 使用するモデル
model_name = "gemini-2.0-flash"  # 動作確認済みのモデル

# Gemini Pro モデルの初期化
gemini_llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=GOOGLE_API_KEY)

def evaluate_accuracy_with_gemini_formatted(question: str, context: str, answer: str):
    """
    Geminiを使ってRAGの回答の正確性を評価し、指定の形式で出力する関数。

    Args:
        question: 元の質問。
        context: 検索されたコンテキスト。
        answer: 生成された回答。

    Returns:
        str: 整形された評価結果（質問、回答、評価、理由）。
    """
    evaluation_prompt = f"""以下の情報に基づいて、生成された回答の正確性を0から4のスケールで評価してください。

    評価スケール：
    0: 全く不正確。回答の全てまたは主要な部分が誤っており、質問に対して全く的外れな情報を提供している。
    1: ほぼ不正確。回答の大部分が不正確であるか、誤った情報に基づいている。わずかに正しい情報が含まれている程度。
    2: 部分的に正確。回答には正確な情報も含まれているが、重要な誤りや不正確な点、または誤解を招く可能性のある記述が含まれている。
    3: ほぼ正確。回答の大部分は正確であり、質問に対して適切に答えている。しかし、細部にわずかな不正確さ、曖昧さ、または重要でない情報の欠落が見られる場合がある。
    4: 完全に正確。回答は質問に対して適切かつ網羅的に答えており、提示された情報に誤りや不確かな点は一切なく、事実に基づいている。コンテキストとも完全に一致している。

    質問: {question}
    コンテキスト: {context}
    回答: {answer}

    評価（0〜4の整数値）：
    理由：
    """

    response = gemini_llm.invoke(evaluation_prompt)
    # Geminiの応答を評価と理由に分割
    parts = response.content.split("理由：", 1)
    evaluation = parts[0].replace("評価（0〜4の整数値）：", "").strip()
    reason = parts[1].strip() if len(parts) > 1 else "理由の抽出に失敗しました。"

    formatted_output = f"""質問：{question}
回答：{answer}
評価：{evaluation}
理由：{reason}
"""
    return formatted_output

# 評価の実行例
question_example = "東京タワーの高さは何メートルですか？"
context_example = "東京タワーは、東京都港区にある電波塔であり、観光名所である。その高さは333メートルである。1958年に完成した。"
answer_example_accurate = "東京タワーの高さは333メートルです。"
answer_example_partially_accurate = "東京タワーの高さは約300メートルです。"
answer_example_inaccurate = "東京タワーはパリにあります。"

print("正確な回答の評価:")
evaluation_accurate_formatted = evaluate_accuracy_with_gemini_formatted(question_example, context_example, answer_example_accurate)
print(evaluation_accurate_formatted)

print("\n部分的に正確な回答の評価:")
evaluation_partially_accurate_formatted = evaluate_accuracy_with_gemini_formatted(question_example, context_example, answer_example_partially_accurate)
print(evaluation_partially_accurate_formatted)

print("\n不正確な回答の評価:")
evaluation_inaccurate_formatted = evaluate_accuracy_with_gemini_formatted(question_example, context_example, answer_example_inaccurate)
print(evaluation_inaccurate_formatted)

正確な回答の評価:
質問：東京タワーの高さは何メートルですか？
回答：東京タワーの高さは333メートルです。
評価：4
理由：回答はコンテキストで提供された情報と完全に一致しており、質問に正確かつ適切に答えているため。


部分的に正確な回答の評価:
質問：東京タワーの高さは何メートルですか？
回答：東京タワーの高さは約300メートルです。
評価：評価: 3

理由: 回答は東京タワーの高さについて概ね正確ですが、正確な高さである333メートルではなく、約300メートルと述べているため、わずかに不正確です。コンテキストには正確な高さが記載されているため、より正確な回答が期待できます。
理由：理由の抽出に失敗しました。


不正確な回答の評価:
質問：東京タワーの高さは何メートルですか？
回答：東京タワーはパリにあります。
評価：0
理由：回答は東京タワーの場所が間違っており（パリではなく東京）、高さに関する情報も提供していないため、全く不正確です。



### 1.3 ベースモデルの評価

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import re  # 正規表現モジュールを追加
import os

# **ここにあなたの Google AI Studio の API キーを入力してください**
# GOOGLE_API_KEY = "YOUR_API_KEY"

# 使用するモデル
model_name = "gemini-2.0-flash"  # 動作確認済みのモデル

# Gemini Pro モデルの初期化
gemini_llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=GOOGLE_API_KEY)

def evaluate_accuracy_with_gemini(question: str, answer: str, model_answer: str):
    """
    Geminiを使ってLLMの回答の正確性を評価する関数（コンテキストなし、質問と模範解答で評価）。

    Args:
        question: 元の質問。
        answer: 生成された回答。
        model_answer: 模範解答。

    Returns:
        tuple: 評価スコア (int) と理由 (str)。評価に失敗した場合は None, エラーメッセージ。
    """
    evaluation_prompt = f"""以下の情報に基づいて、生成された回答の正確性を0から4のスケールで評価してください。

    評価スケール：
    0: 全く不正確。回答の全てまたは主要な部分が誤っており、質問に対して全く的外れな情報を提供している。
    1: ほぼ不正確。回答の大部分が不正確であるか、誤った情報に基づいている。わずかに正しい情報が含まれている程度。
    2: 部分的に正確。回答には正確な情報も含まれているが、重要な誤りや不確かな点、または誤解を招く可能性のある記述が含まれている。
    3: ほぼ正確。回答の大部分は正確であり、質問に対して適切に答えている。しかし、細部にわずかな不正確さ、曖昧さ、または重要でない情報の欠落が見られる場合がある。
    4: 完全に正確。回答は質問に対して適切かつ網羅的に答えており、提示された情報に誤りや不確かな点は一切なく、特に模範解答の内容と照らし合わせて評価してください。

    質問: {question}
    生成された回答: {answer}
    模範解答: {model_answer}

    評価（0〜4の整数値）：
    理由：
    """

    try:
        response = gemini_llm.invoke(evaluation_prompt)
        parts = response.content.split("理由：", 1)
        evaluation_str = parts[0].replace("評価（0〜4の整数値）：", "").replace("評価：", "").strip()
        match = re.search(r'\d+', evaluation_str)
        if match:
            score = int(match.group(0))
        else:
            score = None
        reason = parts[1].strip() if len(parts) > 1 else "理由の抽出に失敗しました。"
        return score, reason
    except Exception as e:
        return None, f"評価エラー: {e}"

# 設定
output_file = datafolder + "llm_responses.json" # LLMの出力先のJSONファイル名
questions_file = datafolder + "Q&A.json" # 元の質問と模範解答のファイル名
result_file = datafolder + "llm_result.json" # 評価結果を格納するファイル名
summary_file = datafolder + "llm_result_summary.json" # 評価概要を格納するファイル名
# datafolder = "" # 必要に応じてデータフォルダのパスを設定

# JSONファイルから質問と回答を読み込む
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        llm_outputs = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{output_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{output_file}' のJSON形式が不正です。")
    exit()

# JSONファイルから元の質問と模範解答を読み込む
try:
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions_data = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{questions_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{questions_file}' のJSON形式が不正です。")
    exit()

evaluation_results = []
total_score = 0
num_evaluated = 0

print("評価結果:")
for i, item in enumerate(llm_outputs):
    if i < len(questions_data):
        original_question = questions_data[i]["質問"]
        model_answer = questions_data[i].get("模範解答", "模範解答が提供されていません")
        llm_answer = item.get("回答", "回答がありません")

        print(f"\n質問番号: {i+1}")
        print(f"質問内容: {original_question}")
        print(f"回答番号: {i+1}")
        print(f"回答内容: {llm_answer}")
        print(f"模範解答: {model_answer}")

        evaluation_result = evaluate_accuracy_with_gemini(original_question, llm_answer, model_answer)

        result_entry = {
            "質問番号": i + 1,
            "質問内容": original_question,
            "回答番号": i + 1,
            "回答内容": llm_answer,
            "模範解答": model_answer
        }

        if evaluation_result[0] is not None:
            score, reason = evaluation_result
            print(f"評価: {score}")
            print(f"評価理由: {reason}")
            result_entry["評価"] = score
            result_entry["評価理由"] = reason
            total_score += score
            num_evaluated += 1
        else:
            print(f"評価エラー: {evaluation_result[1]}")
            result_entry["評価エラー"] = evaluation_result[1]

        evaluation_results.append(result_entry)

    else:
        print(f"\nエラー: 生成された回答の数({len(llm_outputs)})が、元の質問の数({len(questions_data)})を超えています。")
        break

print("\n---")
summary = {}
if num_evaluated > 0:
    average_score = total_score / num_evaluated
    summary["合計点数"] = total_score
    summary["平均値"] = average_score
    print(f"合計点数: {total_score}")
    print(f"平均値: {average_score:.2f}")
else:
    summary["エラー"] = "評価を実行できませんでした。"
    print("評価を実行できませんでした。")

# 評価結果をJSONファイルに保存
try:
    with open(result_file, 'w', encoding='utf-8') as f:
        json.dump(evaluation_results, f, ensure_ascii=False, indent=4)
    print(f"\n評価結果を '{result_file}' に保存しました。")
    if summary:
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=4)
        print(f"評価の概要を '{summary_file}' に保存しました。")

except Exception as e:
    print(f"エラー: 評価結果のJSONファイルへの保存に失敗しました: {e}")

評価結果:

質問番号: 1
質問内容: 令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。
回答番号: 1
回答内容: 令和7年度の防衛費の予算総額は、約10兆円です。主要な内訳は、 * **装備品**:  最新鋭の戦闘機や艦船の購入や整備 * **人材**:  自衛官の給与・手当の増加や、防衛関連の専門家育成 * **運用**:  防衛施設の維持・管理や、防衛システムの運用 * **研究開発**:  新しい技術の開発や、既存技術の改良 * **情報収集**:  敵の動きや状況を把握するための情報収集 * **国際協力**:  他の国と協力して、国際的な安全保障の維持
模範解答: 令和7年度の防衛費の予算総額は、歳出ベースで8兆4,748億円、契約ベースで8兆4,332億円です。

その主要な内訳は以下の通りです。（歳出ベース）

* 装備品費：3兆1,714億円
* 需品費・施設費：1兆4,918億円
* 人件・糧食費等：2兆9,186億円
* 研究開発費：8,176億円
* 施設整備費：7,586億円
* その他：3,168億円
評価: 2
評価理由: * **予算総額:** 生成された回答では「約10兆円」とありますが、模範解答では歳出ベースで「8兆4,748億円」と具体的な金額が記載されており、1.5兆円以上のずれがあります。
* **内訳:** 主要な内訳については、大まかなカテゴリは合っていますが、具体的な金額が記載されていません。また、内訳のカテゴリ分けも模範解答とは異なり、正確性に欠けます。例えば、「需品費・施設費」や「人件・糧食費等」といった具体的な費目がなく、代わりに「人材」や「運用」といった抽象的な表現が用いられています。
* **具体性:** 模範解答では、各内訳の金額が具体的に示されており、回答の信頼性が高まっています。一方、生成された回答では具体的な数値が示されていないため、情報の信頼性が低いと言えます。

以上の点から、生成された回答は部分的に正確であるものの、金額の誤りや具体性の欠如など、重要な問題点を含んでいるため、評価は2と判断します。

質問番号: 2
質問内容: 令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてくださ


わかりにくいので、合計点数と平均点を再度プリントします

In [ ]:
print(f"合計点数: {total_score}")
print(f"平均値: {average_score:.2f}")

合計点数: 3
平均値: 0.60


### 1.4 ベースモデルの結果
**合計点数:3**

**平均値：0.60**

０点で良さそうなところを１〜２点と評価していてやや甘めですが、許容内の評価だと思います。

ベースモデルではほぼ回答できてないことが確認できました。

# 2 参照文書データの活用

## 2.1 参照文書をテキスト化してそのまま活用 （文書整形なし）

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: 参照文書をPyPDF2でテキスト化した文書
* **目的**: モデルに参照文書に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

まずは、初期のRAGの実装（ベーシックアプローチ）として参照文書をPyPDF2で出力したテキストをそのまま使用します。文書の分割方法が以下の通りです。

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 参照文書をPyPDF2で抽出した生テキストをそのまま使用
* **分割方法**: ページ単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

文章のベクトル化に使用するSentenceTransfomerは、「Paper&Hucks Vol.32」で井伊講師がご紹介していた日本語センテンストランスフォーマーである「sarashina-embedding-v1-1b」（SB Intuitions開発）を使用してみました。

In [ ]:
from sentence_transformers import SentenceTransformer

# モデルの読み込み（初回のみDL）
model = SentenceTransformer("sbintuitions/sarashina-embedding-v1-1b")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.57k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/669 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.83M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/968 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

#### 2.1.1 参照文書のベクトル化
テキスト整形なしの参照文書はページごとにチャンクに分けてJSONファイル化しています。（チャンク分割、JSONファイル化は別ノートブック「PDFtoChank.ipynb」にて実施

In [ ]:
import json
# import faiss
import numpy as np
from tqdm import tqdm

pattern1_file = datafolder + "pattern1_raw_pages.json"

# JSONファイルの読み込み
with open(pattern1_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# データ準備
documents = []
metadata = []

for item in raw_data:
    content = item["content"].replace("\n", " ").strip()
    title = item["title"]
    documents.append(content)
    metadata.append({"title": title, "content": content})

# エンベディング生成
documents_embeddings = model.encode(documents, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

#### 2.1.2 質問のembedding

In [ ]:
# JSONファイルから読み込み

question_file = datafolder + 'question.json'

with open(question_file, 'r', encoding='utf-8') as f:
    question_data = json.load(f)

# 質問文だけを抽出
questions = [q["質問"] for q in question_data]

# 質問のベクトル化
# model = SentenceTransformer('all-MiniLM-L6-v2')
question_embeddings = model.encode(questions, normalize_embeddings=True)

#### 2.1.3 質問とドキュメントの類似度の算出

質問とドキュメントのコサイン類似度を計算し、質問ごとに類似度の高い順に３つの文書をJSONファイルに格納します。

ファイル名は

pattern1_similarity.json

です。

In [ ]:
import numpy as np
import json
from sklearn.preprocessing import normalize

top_k = 3
threshold = 0  # 類似度の閾値（任意で調整）

results = []  # 出力用のリスト

# コサイン類似度行列を計算
# その前に正規化
question_embeddings_norm = normalize(question_embeddings)
documents_embeddings_norm = normalize(documents_embeddings)

similarity_matrix = np.matmul(question_embeddings_norm, documents_embeddings_norm.T)

# 各質問に対して類似度上位ドキュメントを取得
for i, question in enumerate(questions):
    top_indices = similarity_matrix[i].argsort()[::-1][:top_k]
    matches = []
    for rank, idx in enumerate(top_indices):
        score = float(similarity_matrix[i][idx])
        if score < threshold:
            continue
        matches.append({
            "rank": rank + 1,
            "score": score,
            "doc_content":documents[idx]
        })
        print("rank:",rank +1)
        print("score:",score)
        print("doc_content:",documents[idx])

    results.append({
        "question": question,
        "matches": matches
    })

# JSONファイルに保存
with open(datafolder + "pattern1_similarity.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("JSONファイル 'pattern1_similarity.json' を保存しました。")

rank: 1
score: 0.9163950681686401
doc_content: 防 衛 省 Ministry of  Defense 防衛力抜本的強化の 進捗と予算 令和７年度予算の概要
rank: 2
score: 0.9145017266273499
doc_content: （単位：億円）単位：億円、％ 〔〕：構成比 歳出化経費の内訳 維持費等 17,769 〔41.2%〕 基地対策経費等 892 〔2.1%〕研究開発費 3,310 〔7.7%〕装備品等購入費 10,575 〔24.5%〕航空機購入費 3,250 〔7.5%〕艦船建造費等 2,713 〔6.3%〕施設整備費等 3,601 〔8.4%〕その他 1,009 〔2.3%〕 歳出化経費 令和７年度予算額 ４３，１１９億円 注１：ＳＡＣＯ関係経費及び米軍再編関係経費のうち地元負担軽減分に係る経費を除く。 注２：令和６年度予算には２０７億円、令和７年度予算には２０１億円のデジタル庁計上分を含む。項目令 和 ６ 年 度 予 算 額令 和 ７ 年 度 予 算 額対前年度 増△減額 維持費等 １６，７３２ １７，７６９ １，０３７ 油購入費 － １ １ 修理費 １５，９２４ １７，０５４ １，１３０ 教育訓練費等 ８０９ ７１３ △９５ 基地対策経費等 ８６９ ８９２ ２３ 研究開発費 １，９５９ ３，３１０ １，３５０ 装備品等購入費 ７，７８３ １０，５７５ ２，７９２ 航空機購入費 ５，２７６ ３，２５０ △２，０２７ 艦船建造費等 １，９８５ ２，７１３ ７２８ 施設整備費等 ２，４９６ ３，６０１ １，１０６ その他（電子計算機等借料 等）８２７ １，００９ １８２ 合 計 ３７，９２８ ４３，１１９ ５，１９１ 64
rank: 3
score: 0.9126102924346924
doc_content: 項目令和６年度 予算額令和７年度 予算額対前年度 増△減額 維持費等 ・油購入費 ・修理費 ・教育訓練費 ・医療費等 ・営舎費等 （光熱水料、燃料費等）７，７５９ １，５７３ ３，６０８ ４７１ ３３３ １，７７４７，４７６ １，２８４ ３，５６４ ５１４ ３６８ １，７４７△２８３ △２８９ △４４ ４３ ３５ △２７ 基地対策経費等 ・基地周辺対策経費 ・同盟強靱化予算 (在

In [ ]:
# このセル使用しない。
# import numpy as np

# # コサイン類似度を計算（質問×ドキュメントのスコア行列）
# similarity_matrix = np.matmul(question_embeddings, doc_embeddings.T)

# # 各質問について類似度上位3件を取得
# top_k = 3
# for i, question in enumerate(questions):
#     print(f"\n--- 質問{i+1}: {question} ---")
#     top_indices = similarity_matrix[i].argsort()[::-1][:top_k]
#     for rank, idx in enumerate(top_indices):
#         print(f"  Top {rank+1}: 類似度={similarity_matrix[i][idx]:.4f}, ドキュメント: {docs[idx]}")

### 2.1.4 初期RAGによる回答の生成
では、得られた類似度トップ３をプロンプトに組み込んで回答を生成します。

生成した回答もJSONファイルに保存します。

ファイル名は**raw_rag_response.json**です。

In [ ]:
# プロンプト生成関数
def build_rag_prompt(question, top_docs):
    prompt = f"以下の質問に対して、与えられた参考情報を使って正確に回答してください。\n\n"
    prompt += f"質問:\n{question}\n\n"
    prompt += "参考情報:\n"
    for i, doc in enumerate(top_docs, 1):
        prompt += f"{i}. {doc['content']}\n"
    prompt += "\n回答:\n"
    return prompt

In [ ]:
## 回答生成関数
def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048).to(device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        temperature=0
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return output_text.split("回答:")[-1].strip()

In [ ]:
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")


response = []

for i, question in tqdm(enumerate(questions), total=len(questions)):
    top_indices = similarity_matrix[i].argsort()[::-1][:3]  # Top 3
    top_docs = [
        {
            "rank": rank + 1,
            "score": float(similarity_matrix[i][idx]),
            "content": documents[idx]
        }
        for rank, idx in enumerate(top_indices)
    ]

    prompt = build_rag_prompt(question, top_docs)
    answer = generate_answer(prompt)

    print(f"\n--- 質問{i+1}: {question} ---")
    print(f"生成された回答:\n{answer}")

    response.append({
        "質問": question,
        # "top_docs": top_docs,
        "回答": answer
    })

# 保存
with open(datafolder + "row_rag_responses.json", "w", encoding="utf-8") as f:
    json.dump(response, f, ensure_ascii=False, indent=2)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 20%|██        | 1/5 [00:08<00:34,  8.58s/it]


--- 質問1: 令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。 ---
生成された回答:
令和7年度の防衛費の予算総額は**43,119億円**です。
その主要な内訳は以下の通りです。

* **維持費等:** 41.3%  (7,476億円)
* **基地対策経費等:** 24.0%  (4,356億円)
* **研究開発費:** 8.9%  (1,619億円)
* **装備品等購入費等:** 12.2%  (2,205億円)
* **施設整備費等:** 3.4%  (619億円)
* **その他:** 10.2%  (1,846億円)


 40%|████      | 2/5 [00:24<00:39, 13.01s/it]


--- 質問2: 令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。 ---
生成された回答:
令和7年度の防衛力整備計画で重視されている7つの分野は以下の通りです。

1. **スタンド・オフ 防衛能力**：〇トマホークの取得1年前倒し、〇島嶼防衛用高速滑空弾の事前発射試験の実施など、海上自衛官等に対する要員養成教育の開始など、海上自衛能力の強化を図っています。
2. **統合防空ミサイル**：イージス・システム搭載艦の2隻の建造契約の締結など、統合防空ミサイルの運用能力を強化しています。
3. **ＧＰＩの日米共同開発**：GPIの日米共同開発に係る開発コンセプトの決定など、日米の連携を強化しています。
4. **無人アセット**：各種無人アセットの運用実証の実施など、無人アセットの運用能力を強化しています。
5. **領域横断作戦能力**：航空自衛隊の第2宇宙システム管理隊の新編など、宇宙・サイバー・陸海空領域における作戦能力の強化を図っています。
6. **指揮統制・情報連携機能**：防衛省クラウドへの集約に向け、各自衛隊クラウドに係るシステム設計・製造に着手など、情報連携機能の強化を図っています。
7. **持続性・強靱性**：全国で12施設における57棟の火薬庫の新設など、弾薬・維持整備・施設の強靱化を図っています。


...


--- 質問3: 過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。 ---
生成された回答:
過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。


**解答:**

日本の防衛費の推移は、過去5年間で、**GDP比で増加傾向**にあります。 

具体的な推移は以下の通りです。

* **令和6年度:**  GDP比で約1.4%増加
* **令和7年度:**  GDP比で約1.5%増加


 80%|████████  | 4/5 [00:54<00:15, 15.48s/it]


--- 質問4: 令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。 ---
生成された回答:
**

令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額は **４，０９７億円** です。

主要項目は以下の通りです。

* **生活・勤務環境の改善**:  
    * 3,878億円：組織文化の改革、営舎内居室の個室化等の推進、庁舎・隊舎の改修や修繕、備品や日用品等の整備など。
    *  **主な改善策**:
        *  自衛官候補生の2士任用時に支給する自衛官任用一時金の引上げ
        *  予備自衛官手当の年額48,000円→ 147,600円
        *  訓練招集手当（予備自


100%|██████████| 5/5 [00:57<00:00, 11.47s/it]


--- 質問5: 令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？ ---
生成された回答:
令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額は？


**解答:**
新型ＦＦＭの建造は３隻で、3,148億円です。


In [ ]:
print(results)

[{'question': '令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。', 'top_docs': [{'rank': 1, 'score': 0.9163951277732849, 'content': '防 衛 省 Ministry of  Defense 防衛力抜本的強化の 進捗と予算 令和７年度予算の概要'}, {'rank': 2, 'score': 0.9145017266273499, 'content': '（単位：億円）単位：億円、％ 〔〕：構成比 歳出化経費の内訳 維持費等 17,769 〔41.2%〕 基地対策経費等 892 〔2.1%〕研究開発費 3,310 〔7.7%〕装備品等購入費 10,575 〔24.5%〕航空機購入費 3,250 〔7.5%〕艦船建造費等 2,713 〔6.3%〕施設整備費等 3,601 〔8.4%〕その他 1,009 〔2.3%〕 歳出化経費 令和７年度予算額 ４３，１１９億円 注１：ＳＡＣＯ関係経費及び米軍再編関係経費のうち地元負担軽減分に係る経費を除く。 注２：令和６年度予算には２０７億円、令和７年度予算には２０１億円のデジタル庁計上分を含む。項目令 和 ６ 年 度 予 算 額令 和 ７ 年 度 予 算 額対前年度 増△減額 維持費等 １６，７３２ １７，７６９ １，０３７ 油購入費 － １ １ 修理費 １５，９２４ １７，０５４ １，１３０ 教育訓練費等 ８０９ ７１３ △９５ 基地対策経費等 ８６９ ８９２ ２３ 研究開発費 １，９５９ ３，３１０ １，３５０ 装備品等購入費 ７，７８３ １０，５７５ ２，７９２ 航空機購入費 ５，２７６ ３，２５０ △２，０２７ 艦船建造費等 １，９８５ ２，７１３ ７２８ 施設整備費等 ２，４９６ ３，６０１ １，１０６ その他（電子計算機等借料 等）８２７ １，００９ １８２ 合 計 ３７，９２８ ４３，１１９ ５，１９１ 64'}, {'rank': 3, 'score': 0.9126102328300476, 'content': '項目令和６年度 予算額令和７年度 予算額対前年度 増△減額 維持費等 ・油購入費 ・修理費 ・教育訓練費 ・医療費等 ・営舎費等 （光熱水料、燃料費等）７，７５９ １，５７３ ３，

### 2.1.5 初期RAGによる回答結果の評価

1.2で説明したGeminiAPIを利用した評価方法で評価します。

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import re  # 正規表現モジュールを追加
import os

# **ここにあなたの Google AI Studio の API キーを入力してください**
GOOGLE_API_KEY = ""

# 使用するモデル
model_name = "gemini-2.0-flash"

# Gemini Pro モデルの初期化
gemini_llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=GOOGLE_API_KEY)

def evaluate_accuracy_with_gemini(question: str, answer: str, model_answer: str):
    """
    Geminiを使ってLLMの回答の正確性を評価する関数（コンテキストなし、質問と模範解答で評価）。

    Args:
        question: 元の質問。
        answer: 生成された回答。
        model_answer: 模範解答。

    Returns:
        tuple: 評価スコア (int) と理由 (str)。評価に失敗した場合は None, エラーメッセージ。
    """
    evaluation_prompt = f"""以下の情報に基づいて、生成された回答の正確性を0から4のスケールで評価してください。

    評価スケール：
    0: 全く不正確。回答の全てまたは主要な部分が誤っており、質問に対して全く的外れな情報を提供している。
    1: ほぼ不正確。回答の大部分が不正確であるか、誤った情報に基づいている。わずかに正しい情報が含まれている程度。
    2: 部分的に正確。回答には正確な情報も含まれているが、重要な誤りや不確かな点、または誤解を招く可能性のある記述が含まれている。
    3: ほぼ正確。回答の大部分は正確であり、質問に対して適切に答えている。しかし、細部にわずかな不正確さ、曖昧さ、または重要でない情報の欠落が見られる場合がある。
    4: 完全に正確。回答は質問に対して適切かつ網羅的に答えており、提示された情報に誤りや不確かな点は一切なく、特に模範解答の内容と照らし合わせて評価してください。

    質問: {question}
    生成された回答: {answer}
    模範解答: {model_answer}

    評価（0〜4の整数値）：
    理由：
    """

    try:
        response = gemini_llm.invoke(evaluation_prompt)
        parts = response.content.split("理由：", 1)
        evaluation_str = parts[0].replace("評価（0〜4の整数値）：", "").replace("評価：", "").strip()
        match = re.search(r'\d+', evaluation_str)
        if match:
            score = int(match.group(0))
        else:
            score = None
        reason = parts[1].strip() if len(parts) > 1 else "理由の抽出に失敗しました。"
        return score, reason
    except Exception as e:
        return None, f"評価エラー: {e}"

# 設定
output_file = datafolder + "row_rag_responses.json" # LLMの出力先のJSONファイル名
questions_file = datafolder + "Q&A.json" # 元の質問と模範解答のファイル名
result_file = datafolder + "row_rag_result.json" # 評価結果を格納するファイル名
summary_file = datafolder + "row_rag_result_summary.json" # 評価概要を格納するファイル名
# datafolder = "" # 必要に応じてデータフォルダのパスを設定

# JSONファイルから質問と回答を読み込む
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        llm_outputs = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{output_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{output_file}' のJSON形式が不正です。")
    exit()

# JSONファイルから元の質問と模範解答を読み込む
try:
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions_data = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{questions_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{questions_file}' のJSON形式が不正です。")
    exit()

evaluation_results = []
total_score = 0
num_evaluated = 0

print("評価結果:")
for i, item in enumerate(llm_outputs):
    if i < len(questions_data):
        original_question = questions_data[i]["質問"]
        model_answer = questions_data[i].get("模範解答", "模範解答が提供されていません")
        llm_answer = item.get("回答", "回答がありません")

        print(f"\n質問番号: {i+1}")
        print(f"質問内容: {original_question}")
        # print(f"回答番号: {i+1}")
        print(f"回答内容: {llm_answer}")
        print(f"模範解答: {model_answer}")

        evaluation_result = evaluate_accuracy_with_gemini(original_question, llm_answer, model_answer)

        result_entry = {
            "質問番号": i + 1,
            "質問内容": original_question,
            "回答番号": i + 1,
            "回答内容": llm_answer,
            "模範解答": model_answer
        }

        if evaluation_result[0] is not None:
            score, reason = evaluation_result
            print(f"評価: {score}")
            print(f"評価理由: {reason}")
            result_entry["評価"] = score
            result_entry["評価理由"] = reason
            total_score += score
            num_evaluated += 1
        else:
            print(f"評価エラー: {evaluation_result[1]}")
            result_entry["評価エラー"] = evaluation_result[1]

        evaluation_results.append(result_entry)

    else:
        print(f"\nエラー: 生成された回答の数({len(llm_outputs)})が、元の質問の数({len(questions_data)})を超えています。")
        break

print("\n---")
summary = {}
if num_evaluated > 0:
    average_score = total_score / num_evaluated
    summary["合計点数"] = total_score
    summary["平均値"] = average_score
    print(f"合計点数: {total_score}")
    print(f"平均値: {average_score:.2f}")
else:
    summary["エラー"] = "評価を実行できませんでした。"
    print("評価を実行できませんでした。")

# 評価結果をJSONファイルに保存
try:
    with open(result_file, 'w', encoding='utf-8') as f:
        json.dump(evaluation_results, f, ensure_ascii=False, indent=4)
    print(f"\n評価結果を '{result_file}' に保存しました。")
    if summary:
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=4)
        print(f"評価の概要を '{summary_file}' に保存しました。")

except Exception as e:
    print(f"エラー: 評価結果のJSONファイルへの保存に失敗しました: {e}")

評価結果:

質問番号: 1
質問内容: 令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。
回答内容: 令和7年度の防衛費の予算総額は**43,119億円**です。
その主要な内訳は以下の通りです。

* **維持費等:** 41.3%  (7,476億円)
* **基地対策経費等:** 24.0%  (4,356億円)
* **研究開発費:** 8.9%  (1,619億円)
* **装備品等購入費等:** 12.2%  (2,205億円)
* **施設整備費等:** 3.4%  (619億円)
* **その他:** 10.2%  (1,846億円)
模範解答: 令和7年度の防衛費の予算総額は、歳出ベースで8兆4,748億円、契約ベースで8兆4,332億円です。

その主要な内訳は以下の通りです。（歳出ベース）

* 装備品費：3兆1,714億円
* 需品費・施設費：1兆4,918億円
* 人件・糧食費等：2兆9,186億円
* 研究開発費：8,176億円
* 施設整備費：7,586億円
* その他：3,168億円
評価: 0
評価理由: 生成された回答は、予算総額と内訳の金額が模範解答と全く異なっており、単位も異なっています。また、内訳の項目も模範解答と大きく異なっており、全体的に誤った情報を提供しているため、全く不正確と評価します。

質問番号: 2
質問内容: 令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。
回答内容: 令和7年度の防衛力整備計画で重視されている7つの分野は以下の通りです。

1. **スタンド・オフ 防衛能力**：〇トマホークの取得1年前倒し、〇島嶼防衛用高速滑空弾の事前発射試験の実施など、海上自衛官等に対する要員養成教育の開始など、海上自衛能力の強化を図っています。
2. **統合防空ミサイル**：イージス・システム搭載艦の2隻の建造契約の締結など、統合防空ミサイルの運用能力を強化しています。
3. **ＧＰＩの日米共同開発**：GPIの日米共同開発に係る開発コンセプトの決定など、日米の連携を強化しています。
4. **無人アセット**：各種無人アセットの運用実証の実施など、無人アセットの運用能力を強化しています。


###2.1.5 テキスト整形なしRAG（初期RAG）の結果

テキストの整形なしでRAGを使いましたが、スコアが向上しませんでした。なぜかは最後に考察します。



In [ ]:
print(f"合計点数: {total_score}")
print(f"平均値: {average_score:.2f}")

合計点数: 3
平均値: 0.60


合計点数: 3
平均値: 0.60

## 2.2 テキスト整形による品質改善検証（品質改善？RAG)


　DAY３の講義において、データをきれいにすることが重要であるとの言及があったことから、さらなる精度向上を目指して、テキストファイルの整形を試みました。

　手動でテキストをきれいにするのは手間ですので正規表現などを用いて自動で整形しました。（別ファイルPDFtoChank.ipynbにて実施）
整形の方針は、無駄な空白を除く、無駄な改行を除く、必要な改行を入れる。の３点を実施しました。

### 2.2.1 テキスト整形した参照文書のベクトル化


参照文書のテキストの整形を行い、ページごとにチャンクに分けてJSONファイル化しています。（チャンク分割、JSONファイル化は別ノートブック「PDFtoChank.ipynb」にて実施）
これを、整形なしの文書と同様、ベクトル化します。
embeddingのモデルは同様に「sarashina-embedding-v1-1b」




In [ ]:
from sentence_transformers import SentenceTransformer

# モデルの読み込み（初回のみDL）
model = SentenceTransformer("sbintuitions/sarashina-embedding-v1-1b")

In [ ]:
import json
# import faiss
import numpy as np
from tqdm import tqdm

pattern1_file = datafolder + "pattern2_cleand_pages.json"

# JSONファイルの読み込み
with open(pattern1_file, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

# データ準備
documents = []
metadata = []

for item in raw_data:
    content = item["content"].replace("\n", " ").strip()
    title = item["title"]
    documents.append(content)
    metadata.append({"title": title, "content": content})

# エンベディング生成
documents_embeddings = model.encode(documents, show_progress_bar=True, convert_to_numpy=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

### 2.2.2 質問とドキュメントの類似度の算出

質問は同じものを用いるので、次は類似度の算出です。同様にコサイン類似度で算出し、結果をJSONファイルに書き出します。

保存するファイル名は、**pattern2_similarity.json** です。

In [ ]:
import numpy as np
import json
from sklearn.preprocessing import normalize

top_k = 3
threshold = 0  # 類似度の閾値（任意で調整）

results = []  # 出力用のリスト

# その前に正規化
question_embeddings_norm = normalize(question_embeddings)
documents_embeddings_norm = normalize(documents_embeddings)

similarity_matrix = np.matmul(question_embeddings_norm, documents_embeddings_norm.T)

# 各質問に対して類似度上位ドキュメントを取得
for i, question in enumerate(questions):
    top_indices = similarity_matrix[i].argsort()[::-1][:top_k]
    matches = []
    for rank, idx in enumerate(top_indices):
        score = float(similarity_matrix[i][idx])
        if score < threshold:
            continue
        matches.append({
            "rank": rank + 1,
            "score": score,
            "doc_content": documents[idx]
        })
        print("rank:",rank +1)
        print("score:",score)
        print("doc_content:",documents[idx])

    results.append({
        "question": question,
        "matches": matches
    })

# JSONファイルに保存
with open(datafolder + "pattern2_similarity.json", "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("JSONファイル 'pattern2_similarity.json' を保存しました。")

rank: 1
score: 0.9183233976364136
doc_content: 防衛省Ministry of  Defense防衛力抜本的強化の進捗と予算令和７年度予算の概要
rank: 2
score: 0.9132819771766663
doc_content: ○我が国は戦後最も厳しく複雑な安全保障環境に直面しているという基本認識の下、国家防衛戦略 ※1及び防衛力整備計画 ※2に基づき、整備計画期間内の防衛力抜本的強化実現に向け、3年目となる令和7年度においても、引き続き必要かつ十分な予算を確保。 －歳出ベースは、整備計画期間中のこれまでの事業の進捗状況や予算の執行状況も踏まえ、予算額を着実に増額し、整備計画対象経費として ８兆 ４, ７４８億円を計上。 －契約ベースは、5年目の令和9年度に向けて、令和7年度中に着手すべき事業を積み上げ、整備計画対象経費として ８兆 ４, ３３２億円を計上。 （今年度までと合わせると、整備計画の実施に必要な契約額 43.5兆円のうち既計上分は ６２％） ○衛星コンステレーションの構築等によるスタンド・オフ防衛能力の強化など、将来の防衛力の中核となる分野を始めとする、7つの重点分野を推進。可動数向上、弾薬確保とともに、防衛施設の強靱化への投資を引き続き重視。防衛生産・技術基盤の強化を推進。基地周辺対策を推進し、米軍再編を着実に実施。 ○特に、自衛官の現下の厳しい募集状況に鑑み、令和6年10月に設置された関係閣僚会議 ※3において取りまとめられた基本方針を踏まえ、人的基盤の強化に係る施策に迅速に取り組み、自衛官であること、また、自衛官であったことの誇りと名誉を得ることができるような、令和の時代に相応しい処遇を確立していく。 ○15区分の配分額の中できめ細やかに進捗状況を管理。足下の物価高・円安の中、引き続き、経費の精査と装備品の効率的な取得を一層推進。 Ⅱ令和7年度予算～基本的な考え方～4 ※1国家防衛戦略（令和4年12月16日国家安全保障会議決定・閣議決定） ※2防衛力整備計画（令和4年12月16日国家安全保障会議決定・閣議決定） ※3自衛官の処遇・勤務環境の改善及び新たな生涯設計の確立に関する関係閣僚会議
rank: 3
score: 0.9085988402366638
doc_content: 項目令和６年

### 2.2.3 品質改善？RAGによる回答の生成

テキスト整形により品質が改善したと思われるRAGによる回答の生成を行います。

回答結果はJSONファイルに書き出します。
ファイル名は **kai_rag_responses.json**です。

In [ ]:
model_name = "google/gemma-2-2b-jpn-it"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")

results = []

for i, question in tqdm(enumerate(questions), total=len(questions)):
    top_indices = similarity_matrix[i].argsort()[::-1][:3]  # Top 3
    top_docs = [
        {
            "rank": rank + 1,
            "score": float(similarity_matrix[i][idx]),
            "content": documents[idx]
        }
        for rank, idx in enumerate(top_indices)
    ]

    prompt = build_rag_prompt(question, top_docs)
    answer = generate_answer(prompt)

    print(f"\n--- 質問{i+1}: {question} ---")
    print(f"生成された回答:\n{answer}")

    results.append({
        "質問": question,
        # "top_docs": top_docs,
        "回答": answer
    })

# 保存
with open(datafolder + "kai_rag_responses.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
 20%|██        | 1/5 [00:07<00:29,  7.34s/it]


--- 質問1: 令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。 ---
生成された回答:
令和7年度の防衛費の予算総額は８兆４,７４８億円です。
主要な内訳は以下の通りです。
* 維持費等：7,476億円（41.3%）
* 基地対策経費等：4,356億円（24.0%）
* 研究開発費：1,619億円（8.9%）
* 装備品等購入費等：2,205億円（12.2%）
* 施設整備費等：619億円（3.4%）
* その他：1,846億円（10.2%）


 40%|████      | 2/5 [00:19<00:30, 10.28s/it]


--- 質問2: 令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。 ---
生成された回答:
令和7年度の防衛力整備計画で重視されている7つの分野は以下の通りです。

1. **スタンド・オフ防衛能力**
    * トマホークの取得1年前倒し
    * 〇島嶼防衛用高速滑空弾の事前発射試験実施
    * 〇12式地対艦誘導弾能力向上型の発射試験実施
2. **統合防空ミサイル防衛能力**
    * イージス・システム搭載艦2隻の建造契約を締結
    * GPIの日米共同開発に係る開発コンセプトを決定
3. **無人アセット防衛能力**
    * 2023年度中に合計10機種の各種UAVの運用実証を実施
    * 各類無人アセットに関する研究開発を推進
    * 滞空型UAV「MQ-9B(シーガーディアン)」を選定
4. **長期運用型UUV**
5. **12式地対艦誘導弾能力向上型**
6. **公共インフラ整備**
7. **我が国及び同志国の抑止力の向上等のための国際協力**


...


--- 質問3: 過去5年間の日本の防衛費の推移を説明してください。特に、GDP比での変化に着目してください。 ---
生成された回答:
**

*  **GDP比での推移:**  参考情報には、具体的なGDP比での推移は記載されていません。


 80%|████████  | 4/5 [00:46<00:13, 13.72s/it]


--- 質問4: 令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額と主要項目を含めて説明してください。 ---
生成された回答:
**

令和７年度の防衛予算において、自衛官等の給与・手当等の見直しに関する予算額は **４,097億円** です。

主要項目は以下の通りです。

* **生活・勤務環境の改善:**
    * 営舎内個室化等の推進
    * 庁舎・隊舎の改修や修繕
    * 新型艦の居住区の魅力化
    * 宿舎の改修や建替え
    * 職場環境の整備
    *無線LAN環境の拡充
    * 主要艦艇における通信環境の構築
    *  託児事業の充実
    * 教育基盤や隊舎の女性用区画の整備
    * 被服等の整備、更新
    * 糧食の魅力化等
* **新たな生涯設計の確立:**
    * 再就職に向けた職業訓練の充実
    * 再就職先拡充のための広報の強化
    * 就職援護情報ネットワークシステムの導入
    *  SNSやターゲティング広告等の募集広報のデジタル化・オンライン化の推進
    * 地方協力本部体制の充実
    * 自衛官等の採用推進のための広報・募集強化
* **自衛官の処遇改善:**
    *  自衛官候補生の処遇改善
    *  士任用時の自衛官任用一時金の引上げ
    *  不慣れな営舎内生活等の労苦を評価し、指定場所生活調整金(仮称)の新設
    *  予備自衛官等の処遇改善
    *  航空管制官の処遇改善
    *  航空機整備員の処遇改善
    *  特殊作戦隊員の処遇改善
    *  災害派遣等手当の引上げ
    *  サイバー専門部隊等の隊員の処遇改善
    *  医療従事者、募集担当者及び最先任上級曹長の処遇改善
    *  自衛官候補生の処遇改善


100%|██████████| 5/5 [00:50<00:00, 10.19s/it]


--- 質問5: 令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額はいくらですか？ ---
生成された回答:
令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額は？


**解説:**

参考情報から、令和7年度の防衛予算で新たに取得する哨戒機の合計機数と合計金額は、以下の通りです。

* **合計機数:** ２機
* **合計金額:** ４１５億円


### 2.2.4 品質改善？RAGによる回答結果の評価

これまでと同様GeminiAPIによる回答の評価を行います。

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import json
import re  # 正規表現モジュールを追加
import os

# **ここにあなたの Google AI Studio の API キーを入力してください**
GOOGLE_API_KEY = ""

# 使用するモデル
model_name = "gemini-2.0-flash"

# Gemini Pro モデルの初期化
gemini_llm = ChatGoogleGenerativeAI(model=model_name, google_api_key=GOOGLE_API_KEY)

def evaluate_accuracy_with_gemini(question: str, answer: str, model_answer: str):
    """
    Geminiを使ってLLMの回答の正確性を評価する関数（コンテキストなし、質問と模範解答で評価）。

    Args:
        question: 元の質問。
        answer: 生成された回答。
        model_answer: 模範解答。

    Returns:
        tuple: 評価スコア (int) と理由 (str)。評価に失敗した場合は None, エラーメッセージ。
    """
    evaluation_prompt = f"""以下の情報に基づいて、生成された回答の正確性を0から4のスケールで評価してください。

    評価スケール：
    0: 全く不正確。回答の全てまたは主要な部分が誤っており、質問に対して全く的外れな情報を提供している。
    1: ほぼ不正確。回答の大部分が不正確であるか、誤った情報に基づいている。わずかに正しい情報が含まれている程度。
    2: 部分的に正確。回答には正確な情報も含まれているが、重要な誤りや不確かな点、または誤解を招く可能性のある記述が含まれている。
    3: ほぼ正確。回答の大部分は正確であり、質問に対して適切に答えている。しかし、細部にわずかな不正確さ、曖昧さ、または重要でない情報の欠落が見られる場合がある。
    4: 完全に正確。回答は質問に対して適切かつ網羅的に答えており、提示された情報に誤りや不確かな点は一切なく、特に模範解答の内容と照らし合わせて評価してください。

    質問: {question}
    生成された回答: {answer}
    模範解答: {model_answer}

    評価（0〜4の整数値）：
    理由：
    """

    try:
        response = gemini_llm.invoke(evaluation_prompt)
        parts = response.content.split("理由：", 1)
        evaluation_str = parts[0].replace("評価（0〜4の整数値）：", "").replace("評価：", "").strip()
        match = re.search(r'\d+', evaluation_str)
        if match:
            score = int(match.group(0))
        else:
            score = None
        reason = parts[1].strip() if len(parts) > 1 else "理由の抽出に失敗しました。"
        return score, reason
    except Exception as e:
        return None, f"評価エラー: {e}"

# 設定
output_file = datafolder + "kai_rag_responses.json" # LLMの出力先のJSONファイル名
questions_file = datafolder + "Q&A.json" # 元の質問と模範解答のファイル名
result_file = datafolder + "kai_rag_result.json" # 評価結果を格納するファイル名
summary_file = datafolder + "kai_rag_result_summary.json" # 評価概要を格納するファイル名
# datafolder = "" # 必要に応じてデータフォルダのパスを設定

# JSONファイルから質問と回答を読み込む
try:
    with open(output_file, 'r', encoding='utf-8') as f:
        llm_outputs = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{output_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{output_file}' のJSON形式が不正です。")
    exit()

# JSONファイルから元の質問と模範解答を読み込む
try:
    with open(questions_file, 'r', encoding='utf-8') as f:
        questions_data = json.load(f)
except FileNotFoundError:
    print(f"エラー: ファイル '{questions_file}' が見つかりません。")
    exit()
except json.JSONDecodeError:
    print(f"エラー: ファイル '{questions_file}' のJSON形式が不正です。")
    exit()

evaluation_results = []
total_score = 0
num_evaluated = 0

print("評価結果:")
for i, item in enumerate(llm_outputs):
    if i < len(questions_data):
        original_question = questions_data[i]["質問"]
        model_answer = questions_data[i].get("模範解答", "模範解答が提供されていません")
        llm_answer = item.get("回答", "回答がありません")

        print(f"\n質問番号: {i+1}")
        print(f"質問内容: {original_question}")
        # print(f"回答番号: {i+1}")
        print(f"回答内容: {llm_answer}")
        print(f"模範解答: {model_answer}")

        evaluation_result = evaluate_accuracy_with_gemini(original_question, llm_answer, model_answer)

        result_entry = {
            "質問番号": i + 1,
            "質問内容": original_question,
            "回答番号": i + 1,
            "回答内容": llm_answer,
            "模範解答": model_answer
        }

        if evaluation_result[0] is not None:
            score, reason = evaluation_result
            print(f"評価: {score}")
            print(f"評価理由: {reason}")
            result_entry["評価"] = score
            result_entry["評価理由"] = reason
            total_score += score
            num_evaluated += 1
        else:
            print(f"評価エラー: {evaluation_result[1]}")
            result_entry["評価エラー"] = evaluation_result[1]

        evaluation_results.append(result_entry)

    else:
        print(f"\nエラー: 生成された回答の数({len(llm_outputs)})が、元の質問の数({len(questions_data)})を超えています。")
        break

print("\n---")
summary = {}
if num_evaluated > 0:
    average_score = total_score / num_evaluated
    summary["合計点数"] = total_score
    summary["平均値"] = average_score
    print(f"合計点数: {total_score}")
    print(f"平均値: {average_score:.2f}")
else:
    summary["エラー"] = "評価を実行できませんでした。"
    print("評価を実行できませんでした。")

# 評価結果をJSONファイルに保存
try:
    with open(result_file, 'w', encoding='utf-8') as f:
        json.dump(evaluation_results, f, ensure_ascii=False, indent=4)
    print(f"\n評価結果を '{result_file}' に保存しました。")
    if summary:
        with open(summary_file, 'w', encoding='utf-8') as f:
            json.dump(summary, f, ensure_ascii=False, indent=4)
        print(f"評価の概要を '{summary_file}' に保存しました。")

except Exception as e:
    print(f"エラー: 評価結果のJSONファイルへの保存に失敗しました: {e}")

評価結果:

質問番号: 1
質問内容: 令和7年度の防衛費の予算総額はいくらですか？また、その主要な内訳を具体的に説明してください。
回答内容: 令和7年度の防衛費の予算総額は８兆４,７４８億円です。
主要な内訳は以下の通りです。
* 維持費等：7,476億円（41.3%）
* 基地対策経費等：4,356億円（24.0%）
* 研究開発費：1,619億円（8.9%）
* 装備品等購入費等：2,205億円（12.2%）
* 施設整備費等：619億円（3.4%）
* その他：1,846億円（10.2%）
模範解答: 令和7年度の防衛費の予算総額は、歳出ベースで8兆4,748億円、契約ベースで8兆4,332億円です。

その主要な内訳は以下の通りです。（歳出ベース）

* 装備品費：3兆1,714億円
* 需品費・施設費：1兆4,918億円
* 人件・糧食費等：2兆9,186億円
* 研究開発費：8,176億円
* 施設整備費：7,586億円
* その他：3,168億円
評価: 1
評価理由: 生成された回答は、予算総額は模範解答と一致していますが、主要な内訳が大きく異なっています。内訳の費目も金額も割合も、模範解答と照らし合わせると全く異なる情報を提供しており、誤った情報に基づいていると判断できます。したがって、ほぼ不正確と評価しました。

質問番号: 2
質問内容: 令和7年度の防衛力整備計画で重視されている7つの分野は何ですか？ それぞれの分野における具体的な取り組みの例を挙げてください。
回答内容: 令和7年度の防衛力整備計画で重視されている7つの分野は以下の通りです。

1. **スタンド・オフ防衛能力**
    * トマホークの取得1年前倒し
    * 〇島嶼防衛用高速滑空弾の事前発射試験実施
    * 〇12式地対艦誘導弾能力向上型の発射試験実施
2. **統合防空ミサイル防衛能力**
    * イージス・システム搭載艦2隻の建造契約を締結
    * GPIの日米共同開発に係る開発コンセプトを決定
3. **無人アセット防衛能力**
    * 2023年度中に合計10機種の各種UAVの運用実証を実施
    * 各類無人アセットに関する研究開発を推進
    * 滞空型UAV「MQ-9B(シーガーディアン)」を選定
4. **長期運用型UUV*

### 2.2.5 テキスト整形ありのRAG（品質改善？RAG）の結果

テキスト整形の結果、少しは品質が改善しました。テキスト整形が効果を挙げた部分もあるようですが、参考情報から読み取れない回答を回答不可とした部分の配点が功を奏しただけであり、だけですので、必ずしもテキスト整形の効果とも言えないと考えられます。

In [ ]:
print(f"合計点数: {total_score}")
print(f"平均値: {average_score:.2f}")

合計点数: 7
平均値: 1.40


# 3 全体を通じた考察
GeminiAPIの評価の結果以下の通りとなりました。

1.   RAGなし

    **合計点数:3**
    **平均値：0.60**
2.   RAGあり（テキスト整形なし）

    **合計点数:3**
    **平均値:0.60**

3.    RAGあり（テキスト整形なし）

    **合計点数:7**
    **平均値:1.40**

　テキスト整形によるノイズの除去のおかげか、RAGあり（テキスト整形あり）において、７年度予算の総額やGDPの記載のないことなど一部は正確に回答できるようになりました。


　一方で、RAGを行っても回答の精度が大幅に向上しなかったのは、参照文献自体の構成の問題かもしれません。防衛省のみならず、政府機関が公表する資料には表やグラフが多用され、PyPDF2による単純なテキスト化では、表やグラフの文脈を表現しきれなかったものと考えられます。

　事実、質問とのコサイン類似度のトップ３のチャンク参照文書には、回答を含む文書が選ばれないケースがありました。例えば、１番目の防衛予算の総額と内訳を問う質問に対し、総額を含む文書が選ばれましたが、内訳に関する文書は正しく選ばれませんでした。これは、元文書においては、内訳の部分が表や円グラフで示されており、この部分が適切にテキスト化されていませんでした。




# 4 さらに改善させるための考察
#### 考察１　表やグラフの読み取り精度を上げる
  　3で考察した通り、表やグラフを文脈にそってテキスト化することがRAGの精度を改善させるために必要であると考えます。これは、講義の中でお話のあったデータの質をあげるというアプローチの１つであり、表がグラフのある文書をRAGの参照文書とする際には避けては通れない事項であると考えます。

  　Geminiに参照文書を読ませて質問、模範回答を生成させた際、グラフや表に含まれる内容も正確に回答していました。マルチモーダルなLLMなどでテキスト化すれば精度の高い表やグラフの読み取りさせるのも、一つの手段となりうると考えられます。

#### 考察２　RAGに用いる文書の選定
  　一方で、テキスト化により文脈が失われる表やグラフを多用した文書ではなく、テキスト中心の文書をRAGの参照文書とするというアプローチもあると思います。

#### 考察３　チャンクの分割の工夫
  　今回は期限が短いこともあり、ページによるチャンク分割を行いましたが、目次の項目ごとにチャンクを分けることにより、必要なチャンクをLLMに渡すことができ精度が向上するものと考えられます。
  　その検証として、目次から抽出した目次の項目ごとにチャンクを分割することを自動化し、その上でRAGの参考文書にすることを試みたのですが、チャンク分割が自動でうまく分けられずに諦めました。
  精度を見る上では手動で印をつけてチャンク分けすることも可能ですので、機会があればこの方法も試したいと思います。

